## Name: Vijay Rakshit  (ID: 12041740), Ansh Azad 12040240

## Graph Code

In [ ]:
class Vertex:
    def __init__(self, node):   ##A vetex class used to store nodes of the graph and its attributes
        self.id = node
        self.label=None   #initially the whole graph is unlabelled
        self.adjacent = {}
        self.keywords = None
        self.tags= None
        self.bag_vector = None
        self.TFIDF_vector = None
        self.degreeCentrality= None
    
     #prints nodes and its neighbours
    def __str__(self):     
        return str(self.id) + ' adjacent: ' + str([x.id for x in self.adjacent])
    
    #function to add neighbours to a node
    def add_neighbor(self, neighbor, weight=0):
        self.adjacent[neighbor] = weight
    
    #returns all the neighbours links
    def get_connections(self):
        return self.adjacent.keys()  

    #returns the node id
    def get_id(self):
        return self.id
    
    #returns the weight of the edge between the node and its particular neighbour
    def get_weight(self, neighbor):
        return self.adjacent[neighbor]
    
    #adds the keywords to the node
    def add_Keywords_tags(self,keywords,tags):
        self.keywords=keywords
        self.tags=tags
    
    #adds the features to the node
    def add_features(self, X, Y):
        self.bag_vector = X
        self.TFIDF_vector = Y
    
    #function to print the feature vectors of the node
    def get_features(self):
        print("Feature 1: Bag of words feature vector")
        print(self.bag_vector)
        print("Feature 2: TF-IDF feature vector")
        print(self.TFIDF_vector)
    
    #function to return the degree centrality of the node
    def get_DegreeCentrality(self,total):
        l=list(self.adjacent.keys())
        x=len(l)
        return x/(total-1)
    
    #funtion to return the clustering coefficient of the node
    def get_Clustering_coefficient(self):
        l=list(self.adjacent.keys())
        c=0
        x=len(l)
        if x==0 or x==1:
            return 0
        else:
            for i in range(x):
                for j in range(x-1):
                    if l[i] in (l[j+1].adjacent.keys()):
                        c+=1
            return c/(x*(x-1))
                    

class Graph:   #class of the graph
    def __init__(self): 
        
        #dictionary storing the links and the vertex corresponding to the link
        
        self.vert_dict = {}  
        self.num_vertices = 0   #stores the number of vertices

    def __iter__(self):    
        
        #returns an iterable to parse through the graph
        return iter(self.vert_dict.values())

    #adds a vertex to the graph which is stored in the dictionary. it takes the link as input
    def add_vertex(self, link):  
        self.num_vertices = self.num_vertices + 1 
        new_vertex = Vertex(link)
        self.vert_dict[link] = new_vertex
        #return new_vertex
    
    #adds the NLP features to the node. It takes the text and link as input to calculate nlp features
    def add_feature(self,link,text):
        node=self.vert_dict[link]
        words=(remove_stop_words(tokenize(text)))
        keywords,tags= keywords_tags(text,words)   #extracting keywords and tags using the function defined below
        node.add_Keywords_tags(keywords,tags)   #adding them to our node
        
        #Calling NLP_feature_Vectors function defined below
        X,Y= NLP_Feature_Vectors(text) 
        node.add_features(X,Y)    #adding the features to our node
        return None

    #returns the vertex corresponding to the link n
    def get_vertex(self, n):
        if n in self.vert_dict:
            return self.vert_dict[n]
        else:
            print ("Vertex not found")
            return None

    #adds an edge between two links (edges are kept track of by each individual nodes adjacency list)
    def add_edge(self, frm_link, to_link, cost = 0):
        if frm_link not in self.vert_dict.keys():
            self.add_vertex(frm_link)
        if to_link not in self.vert_dict.keys():
            self.add_vertex(to_link)

        self.vert_dict[frm_link].add_neighbor(self.vert_dict[to_link], cost)
        self.vert_dict[to_link].add_neighbor(self.vert_dict[frm_link], cost)

    #returns a list of all the vertices
    def get_vertices(self):
        return self.vert_dict.keys()
    
    def Degree_Centrality(self):
        for link,vertex in self.vert_dict.items():
            print(f"{link} \t {vertex.get_DegreeCentrality(self.num_vertices)}")
        return None
    
    def BFS(self):        #graph traversal algorithm
        open_queue=[]
        closed=[]
        
        #starts bfs from the first node in the graph
        for link,vertex in self.vert_dict.items():
            if((link not in open_queue) or (link not in closed)):
                open_queue.append(link)
                neighbours=list(vertex.adjacent.keys())
                for j in neighbours:
                    if((j not in open_queue) or (j not in closed)):
                        open_queue.append(j)
                closed.append(open_queue.pop(open_queue.index(link)))
        return closed
    
    #function returns clustering coefficient of the nodes
    def Clustering_coefficient(self):
        for link,vertex in self.vert_dict.items():
            print(f"{link} \t {vertex.get_Clustering_coefficient()}")
        


## Natural Language Processing

In [ ]:
pip install nltk

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.tokenize import sent_tokenize

In [ ]:
nltk.download('stopwords')

In [ ]:
stop_words = set(stopwords.words('english'))
print(stop_words)

In [ ]:
def tokenize(text):  #gets the raw text extracted from wikipedia and converts it to a list of words 
    return word_tokenize(text)

In [ ]:
#Example text used for testing the code
text="hello! this is the assignment i am doing right now "
nltk.download('punkt')
print(tokenize(text))   

In [ ]:
def remove_stop_words(words): #takes the list of words and removes the stop words and words with length less than 2
    l=[word for word in words if (word not in stop_words)]
    l1=[word for word in l if len(word)>2]
    return l1

In [ ]:
print(remove_stop_words(tokenize(text)))

In [ ]:
from operator import itemgetter
import math

def TF(words):  #sending tokenized and stop word free list of words
    no_of_words = len(words)
#     print(no_of_words)
    tf_score = {}
    for each_word in words:
        if each_word in tf_score:
            tf_score[each_word] += 1
        else:
            tf_score[each_word] = 1
    # Dividing by no_of_words for each dictionary element
    tf_score.update((x, y/int(no_of_words)) for x, y in tf_score.items())
    return (tf_score)

def check_sent(word, sentences):        #returns the number of sentences the word is in
    final = [all([w in x for w in word]) for x in sentences] 
    sent_len = [sentences[i] for i in range(0, len(final)) if final[i]]
    return int(len(sent_len))

def IDF(text,words):                    #takes the raw text and the words list
    sentences = sent_tokenize(text)
    no_of_sentences = len(sentences)
#     print(no_of_sentences)
    idf_score = {}
    for each_word in words:
        if each_word in idf_score:
            idf_score[each_word] = check_sent(each_word, sentences)
        else:
            idf_score[each_word] = 1

    # Performing a log and divide
    idf_score.update((x, math.log(int(no_of_sentences)/y)) for x, y in idf_score.items())

    return(idf_score)
    
def keywords_tags(text,words):   #function that extracts keywords and tags and returns it
    tf_score=TF(words)
    idf_score=IDF(text,words)
    tf_idf_score = {key: tf_score[key] * idf_score.get(key, 0) for key in tf_score.keys()}
    print(tf_idf_score)
    keywords = dict(sorted(tf_idf_score.items(), key = itemgetter(1), reverse = True)[:10])
    tags= dict(sorted(tf_idf_score.items(), key = itemgetter(1), reverse = True)[:4])
    return (list(keywords.keys())),(list(tags.keys()))

In [ ]:
#Example text used for testing keyword and tag extraction code
text="hello! this is the assignment i am doing right now "
words=(remove_stop_words(tokenize(text)))
X,Y=(keywords_tags(text,words))
print(X)
print(Y)

## Feature vector generation- Bag of words vector, TF-IDF vector

In [ ]:
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer


def NLP_Feature_Vectors(text):   #Code taken from the NLP tutorial done in class
    ps = PorterStemmer()
    wordnet=WordNetLemmatizer()
    sentences = nltk.sent_tokenize(text) #Tokenizing 
    corpus = []
    for i in range(len(sentences)):
        review = re.sub('[^a-zA-Z]', ' ', sentences[i]) 
        review = review.lower() #Lowering the sentence 
        review = review.split() #Splitting it into words
        review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))] #Applying stemming
        review = ' '.join(review)
        corpus.append(review)    
        
    # Creating Bag of Words model
    from sklearn.feature_extraction.text import CountVectorizer
    cv = CountVectorizer(max_features = 10)
    X = cv.fit_transform(corpus).toarray()  #bag of words feature vector
    
    # Creating TF-IDF model
    from sklearn.feature_extraction.text import TfidfVectorizer
    
     #Since we have extracted less text we are using less features=20
    tv = TfidfVectorizer(max_features=20)
    
    Y = tv.fit_transform(corpus).toarray() #TF-IDF feature vector
    sum1=np.zeros(len(Y[0]))
    for i in Y:
        sum1+=i
    sum1/=len(Y)
    
    
#     print(len(sentences)) #Checking the number of sentences in the paragraph 
#     print(X) # Printing the final feature matrix 
#     print(X.shape) # Shape of the feature matrix 
#     print(corpus) # Printing the processed text 
#     print(sentences) # Printing the original text
#     print(Y)
#     print(Y.shape)
    return X,sum1   #returning bag of words feature vertor and TFIDF feature vector

## Reading data

In [ ]:
!unzip "/content/b_bit.zip"

Archive:  /content/b_bit.zip
replace b_bit.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import pandas as pd
df=pd.read_csv("b_bit.csv")   #file created after webscraping
df.head()

In [ ]:
df.drop(columns=["out_links"])
df.head()

In [ ]:
df['link'].shape

## Creating the graph

In [ ]:
g=Graph()

In [ ]:
for i in range(len(df['title'])):   #The output of this block is not important
    link=df['link'][i]
    text=df['article'][i]
    out=eval(df['out_link'][i])
    g.add_vertex(link)
    g.add_feature(link,text)
    for j in out:
        g.add_edge(link,j)

{'encyclopedia': 0.00476201521138443, 'brethren': 0.003868165545959774, 'purity': 0.004081727324043797, 'arabic': 0.0007736331091919549, 'رسائل': 0.008155784837357034, 'إخوان': 0.008155784837357034, 'الصفا': 0.008155784837357034, 'also': 0.0, 'variously': 0.008155784837357034, 'known': 0.008155784837357034, 'epistles': 0.000754434140931332, 'sincerity': 0.0013605757746812657, 'loyal': 0.008155784837357034, 'friends': 0.008155784837357034, 'islamic': 0.00036823231373266156, 'treatises': 0.008155784837357034, "rasā'il": 0.008155784837357034, 'written': 0.0011604496637879324, 'mysterious': 0.0016804946233168051, 'basra': 0.0007736331091919549, 'iraq': 0.007818493619652853, 'sometime': 0.00024548820915510765, 'second': 0.008155784837357034, 'half': 0.008155784837357034, 'century': 0.0020408636620218986, 'possibly': 0.008155784837357034, 'later': 0.0, 'great': 0.008155784837357034, 'influence': 0.008155784837357034, 'intellectual': 0.008155784837357034, 'leading': 0.008155784837357034, 'lig

In [ ]:
x=g.vert_dict['https://en.wikipedia.org/wiki/List_of_statistics_journals']  #a single node in the graph

In [ ]:
print(x)   #string representation of the node which gives us the node link and the links attached(adjacent) to this node

In [ ]:
type(x.adjacent)

In [ ]:
x.get_features()   #feature vectors of the node x defined above

In [ ]:
print(x.keywords)  #keywords of the above node

In [ ]:
print(x.tags)  #tags of the above node

In [ ]:
len(x.TFIDF_vector)

In [ ]:
x.bag_vector

# Bfs traversal on whole graph starting from the first node

In [ ]:
l=g.BFS()   

In [ ]:
for i in l:    #BFS order of the nodes
    print("{}:{}".format((l.index(i)+1),i))

In [ ]:
len(l)  #number of nodes connected to the first node which were explored with BFS

In [ ]:
g.Degree_Centrality()  #Degree Centrality of each node

In [ ]:
g.Clustering_coefficient()   #Clustering coefficient of each node

## Visualizing the Graph

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt  #Visualization of the graph using networkx library

In [ ]:
class GraphVisualization:    #Code taken from Gfg for vizualization
   
    def __init__(self):
          
        # visual is a list which stores all 
        # the set of edges that constitutes a
        # graph
        self.visual = []
          
    # addEdge function inputs the vertices of an
    # edge and appends it to the visual list
    def addEdge(self, a, b):
        temp = [a, b]
        self.visual.append(temp)
          
    # In visualize function G is an object of
    # class Graph given by networkx G.add_edges_from(visual)
    # creates a graph with a given list
    # nx.draw_networkx(G) - plots the graph
    # plt.show() - displays the graph
    def visualize(self):
        G = nx.Graph()
        G.add_edges_from(self.visual)
        nx.draw_networkx(G,with_labels=False)
        plt.figure(figsize=(1000,1000))
        plt.show()

In [ ]:
Viz_G=GraphVisualization()
for vertex in g.vert_dict.values():
    l=vertex.adjacent
    for i in l:
        Viz_G.addEdge(vertex.id,i.id)
Viz_G.visualize()

## Labelling the graph according to the given dataset

In [ ]:
df2=pd.read_csv("labeled.csv")  #reading the data from the labeled.csv file

In [ ]:
df2.head()

In [ ]:
def feature(node,link,text):   #A function feature created outside of the graph class (same utility)
        words=(remove_stop_words(tokenize(text)))
        keywords,tags= keywords_tags(text,words)
        node.add_Keywords_tags(keywords,tags)
        X,Y= NLP_Feature_Vectors(text)
        node.add_features(X,Y)
        return None

In [ ]:
label_lst=[]   #list of labelled nodes
len(df2["Link"])   #number of links in the dataframe=464
c=0
for i in range(464):
    c+=1
    label=df2["Label"][i]        #label of each link
    link=df2["Link"][i]          #link of each link
    text=df2["article"][i]       #text in each link
    node=Vertex(link)            #Creating a node of the link
    feature(node,link,str(text))
    node.label=label            #adding the labels to these labelled nodes
    label_lst.append(node)
print(c)

In [ ]:
len(label_lst)  #464 labelled nodes

In [ ]:
len(label_lst[3].TFIDF_vector)

In [ ]:
label_lst[1].label

In [ ]:
label_lst[0].TFIDF_vector.shape

In [ ]:
label_lst[0].TFIDF_vector

# Part D: Label Propogation Algorithm


In [ ]:
for i in g.vert_dict.values():   #nodes from the graph
    maxi=0
    k1=i.TFIDF_vector
    for j in label_lst:    #nodes from the labelled list
        k2=j.TFIDF_vector
        if len(k1)==len(k2):
            dot=[k1[i] * k2[i] for i in range(len(k1))]
            print(dot)
            dotp=sum(dot)
            print(dotp)
            if(dotp>maxi):
                i.label=j.label
                maxi=dotp
        else:
            pass
            
        

In [ ]:
#printing the newly labelled nodes from the unlabelled graph 
c=0
for link,vertex in g.vert_dict.items():
    if(vertex.label!=None):
        c+=1
        print(link,vertex.label)

In [ ]:
t=g.num_vertices
print("number of newly labelled nodes: ",c)
print("Total number of nodes: ",t)
print("Unlabelled nodes: ",(t-c))

## Part E: Graph Traversal and Article Ordering Algorithm 

## Article Ordering Using DFS

In [ ]:
def label_DFS(node,closed):   #sub graph traversal algorithm using labels
    neighbours=list(node.adjacent.keys())
    open_queue=[]
    closed=closed
    k=[]
    for vertex in neighbours:
        if((vertex not in open_queue) and (vertex not in closed)):
            open_queue.append(vertex)
#             open_queue.append((vertex.label,vertex.id,vertex))   #storing a tuple of (label,link,node object)

    for i in open_queue:
#         print(i[0])
#         break
        if i.label==0 or i.label==None:    #removing the unlabelled and irrelevant nodes
            pass
        else:
            k.append(i)
#     print(k)
    sk = sorted(k,key=lambda x:x.label)   #sorting from lowest to highest difficulty
#     print(sk)
    if (len(sk)==0):   #if all the nodes were having either 0 or None labels and were removed before
        sk.append(open_queue[1])
    closed.append(sk[0])
    if len(closed)==10:     #breaking recursive call when 10 nodes are achieved
        return closed
    else:
        return label_DFS(sk[0],closed)

In [ ]:
#Code to get article order via user input (DFS)
flag = 0
node=None
inp=input("Enter a topic: ")
inp_link = "https://en.wikipedia.org/wiki/"+inp
for link,vertex in g.vert_dict.items():
    if inp_link==link:
        node=vertex
#         nlst=list(vertex.adjacent.keys())
        flag+=1
if flag==0:
    print("topic not found")
else:
    closed=[]
    l=label_DFS(node,closed)
    print(l)

for i in l:    #The top 10 articles to read after the input topic according to toughness level
    print(i.id,i.label)    

In [ ]:
for i in l:    #The top 10 articles to read after the input topic according to toughness level
    print(i.id,i.label)

## Article Ordering Using BFS

In [ ]:
def label_BFS(node,closed):   #sub graph traversal algorithm using labels
    neighbours=list(node.adjacent.keys())
    open_queue=[]
    closed=closed
    k=[]
    for vertex in neighbours:
        if((vertex not in open_queue) and (vertex not in closed)):
            open_queue.append(vertex)
#             open_queue.append((vertex.label,vertex.id,vertex))   #storing a tuple of (label,link,node object)

    for i in open_queue:
#         print(i[0])
#         break
        if i.label==0 or i.label==None:    #removing the unlabelled and irrelevant nodes
            pass
        else:
            k.append(i)
#     print(k)
    sk = sorted(k,key=lambda x:x.label)   #sorting from lowest to highest difficulty
    if (len(sk)==0):         #if all the nodes were having either 0 or None labels and were removed before
        sk.append(open_queue[0])
#     print(sk)
    if(len(sk)>10):      #if the length of the sorted list of nodes is more than 10 we take the first 10 nodes
        closed=sk[0:10]
    else:               #else we take the number of nodes we require
        k=len(closed)
        closed+=sk[0:[10-k]]
    if len(closed)==10:     #breaking recursive call when 10 nodes are achieved
        return closed
    else:
        return label_BFS(sk[0],closed)   

In [ ]:
#Code to get article order via user input (DFS)
flag = 0
node=None
inp=input("Enter a topic: ")
inp_link = "https://en.wikipedia.org/wiki/"+inp
for link,vertex in g.vert_dict.items():
    if inp_link==link:
        node=vertex
#         nlst=list(vertex.adjacent.keys())
        flag+=1
if flag==0:
    print("topic not found")
else:
    closed=[]
    l=label_BFS(node,closed)
    print(l)



In [ ]:
for i in l:    #The top 10 articles to read after the input topic according to toughness level
    print(i.id,i.label)